In [21]:
import numpy as np
from scipy import stats
from scipy.stats import norm

In [25]:
def count_s(p1, n1, p2, n2):
    P = (p1*n1+p2*n2)/(n1+n2)
    return np.sqrt(P*(1-P)*(1/n1+1/n2))

def count_p(p1, n1, p2, n2):
    S = count_s(p1, n1, p2, n2)
    t = (p1 - p2)/S
    n = norm(0, 1)
    return 2*(1-stats.norm.cdf(abs(t)))

In [31]:
def proportions_diff_z_stat_ind(p1, n1, p2, n2): 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [34]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [44]:
proportions_diff_z_test(proportions_diff_z_stat_ind(p1, n1, p2, n2), alternative = "greater")

0.37293045872523534

In [26]:
p1 = 10/34
p2 = 4/16
n1 = 34
n2 = 16

In [27]:
count_p(p1, n1, p2, n2)

0.7458609174504707

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [12]:
data = pd.read_csv("banknotes.txt", sep = "\t", header = 0)

In [13]:
data

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


In [38]:
target = data["real"]
data.drop(columns="real", inplace=True)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size= 0.25, random_state = 1)

In [46]:
m1 = LogisticRegression()
m2 = LogisticRegression()
col1 = ["X1", "X2", "X3"]
col2 = ["X4", "X5", "X6"]

In [47]:
m1.fit(X_train[col1], y_train)
m2.fit(X_train[col2], y_train)
a1 = m1.predict(X_test[col1])
a2 = m2.predict(X_test[col2])

/Users/maxim/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/maxim/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [51]:
er1 = np.zeros(a1.shape[0])
er1[a1 != y_test] = 1

er2 = np.zeros(a2.shape[0])
er2[a2 != y_test] = 1

In [52]:
er1.mean()

0.2

In [53]:
er2.mean()

0.02

In [57]:
er1[(er1 == 1) & (er2 == 0)].shape[0]

10

In [58]:
def count_both(er1, er2):
    f = er1[(er1 == 1) & (er2 == 0)].shape[0]
    g = er1[(er1 == 0) & (er2 == 1)].shape[0]
    z = (f - g)/np.sqrt(f+g-(f-g)**2/er1.shape[0])
    return 2*(1 - stats.norm.cdf(abs(z)))

In [59]:
count_both(er1, er2)

0.0032969384555543435

In [61]:
proportions_diff_z_test(proportions_diff_z_stat_ind(er1.mean(), er1.shape[0], er2.mean(), er2.shape[0]))

0.004022237272055307

In [71]:
def find_interval(er1, er2, alpha = 0.05):
    z = stats.norm.ppf(1- alpha/2)
    f = er1[(er1 == 1) & (er2 == 0)].shape[0]
    g = er1[(er1 == 0) & (er2 == 1)].shape[0]
    
    s = np.sqrt(f+g-(f-g)**2/er1.shape[0])
    
    left = (f-g -z*s)/er1.shape[0]
    right = (f-g +z*s)/er2.shape[0]
    return left, right

In [72]:
find_interval(er1, er2)

(0.05994520627961433, 0.3000547937203856)

In [75]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [76]:
proportions_diff_confint_ind(er1, er2)

(0.0625328978652606, 0.2974671021347394)

In [84]:
t = (541.5 - 525)/(100/np.sqrt(100))

In [85]:
norm = stats.norm(0, 1)

In [86]:
(1- norm.cdf(t))

0.0494714680336481